## 1. The NIST Special Publication 800-63B
<p>If you – 50 years ago – needed to come up with a secret password you were probably part of a secret espionage organization or (more likely) you were pretending to be a spy when playing as a kid. Today, many of us are forced to come up with new passwords <em>all the time</em> when signing into sites and apps. As a password <em>inventeur</em> it is your responsibility to come up with good, hard-to-crack passwords. But it is also in the interest of sites and apps to make sure that you use good passwords. The problem is that it's really hard to define what makes a good password. However, <em>the National Institute of Standards and Technology</em> (NIST) knows what the second best thing is: To make sure you're at least not using a <em>bad</em> password. </p>
<p>In this notebook, we will go through the rules in <a href="https://pages.nist.gov/800-63-3/sp800-63b.html">NIST Special Publication 800-63B</a> which details what checks a <em>verifier</em> (what the NIST calls a second party responsible for storing and verifying passwords) should perform to make sure users don't pick bad passwords. We will go through the passwords of users from a fictional company and use R to flag the users with bad passwords. But us being able to do this already means the fictional company is breaking one of the rules of 800-63B:</p>
<blockquote>
  <p>Verifiers SHALL store memorized secrets in a form that is resistant to offline attacks. Memorized secrets SHALL be salted and hashed using a suitable one-way key derivation function.</p>
</blockquote>
<p>That is, never save users' passwords in plaintext, always encrypt the passwords! Keeping this in mind for the next time we're building a password management system, let's load in the data.</p>
<p><em>Warning: The list of passwords and the fictional user database both contain <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have not been filtered in any way and include words that are explicit, derogatory and offensive.</em></p>

In [20]:
# Importing the tidyverse library
library(tidyverse)

# Loading in datasets/users.csv 
users <- read_csv('datasets/users.csv')

# Counting how many users we've got
nrow(users)

# Taking a look at the 12 first users
head(users, 12)

Parsed with column specification:
cols(
  id = col_double(),
  user_name = col_character(),
  password = col_character()
)


[1] 982

id,user_name,password
<dbl>,<chr>,<chr>
1,vance.jennings,joobheco
2,consuelo.eaton,0869347314
3,mitchel.perkins,fabypotter
4,odessa.vaughan,aharney88
5,araceli.wilder,acecdn3000
6,shawn.harrington,5278049
7,evelyn.gay,master
8,noreen.hale,murphy
9,gladys.ward,lwsves2


## 2. Passwords should not be too short
<p>If we take a look at the first 12 users above we already see some bad passwords. But let's not get ahead of ourselves and start flagging passwords <em>manually</em>. What is the first thing we should check according to the NIST Special Publication 800-63B?</p>
<blockquote>
  <p>Verifiers SHALL require subscriber-chosen memorized secrets to be at least 8 characters in length.</p>
</blockquote>
<p>Ok, so the passwords of our users shouldn't be too short. Let's start by checking that!</p>

In [22]:
# Calculating the lengths of users' passwords
users$length <- str_length(users$password)

# Flagging the users with too short passwords
users$too_short <- ifelse(users$length < 8, TRUE, FALSE)

# Counting the number of users with too short passwords
sum(users$too_short)

# Taking a look at the 12 first rows
head(users, 12)

[1] 376

id,user_name,password,length,too_short
<dbl>,<chr>,<chr>,<int>,<lgl>
1,vance.jennings,joobheco,8,FALSE
2,consuelo.eaton,0869347314,10,FALSE
3,mitchel.perkins,fabypotter,10,FALSE
4,odessa.vaughan,aharney88,9,FALSE
5,araceli.wilder,acecdn3000,10,FALSE
6,shawn.harrington,5278049,7,TRUE
7,evelyn.gay,master,6,TRUE
8,noreen.hale,murphy,6,TRUE
9,gladys.ward,lwsves2,7,TRUE


## 3.  Common passwords people use
<p>Already this simple rule flagged a couple of offenders among the first 12 users. Next up in Special Publication 800-63B is the rule that</p>
<blockquote>
  <p>verifiers SHALL compare the prospective secrets against a list that contains values known to be commonly-used, expected, or compromised.</p>
  <ul>
  <li>Passwords obtained from previous breach corpuses.</li>
  <li>Dictionary words.</li>
  <li>Repetitive or sequential characters (e.g. ‘aaaaaa’, ‘1234abcd’).</li>
  <li>Context-specific words, such as the name of the service, the username, and derivatives thereof.</li>
  </ul>
</blockquote>
<p>We're going to check these in order and start with <em>Passwords obtained from previous breach corpuses</em>, that is, websites where hackers have leaked all the users' passwords. As many websites don't follow the NIST guidelines and encrypt passwords there now exist large lists of the most popular passwords. Let's start by loading in the 10,000 most common passwords which I've taken from <a href="https://github.com/danielmiessler/SecLists/tree/master/Passwords">here</a>.</p>

In [24]:
# Reading in the top 10000 passwords
common_passwords <- read_lines('datasets/10_million_password_list_top_10000.txt')

# Taking a look at the top 100
head(common_passwords, 100)

Parsed with column specification:
cols(
  `123456` = col_character()
)


123456
<chr>
password
12345678
qwerty
123456789
12345
1234
111111
1234567
dragon


## 4.  Passwords should not be common passwords
<p>The list of passwords was ordered, with the most common passwords first, and so we shouldn't be surprised to see passwords like <code>123456</code> and <code>qwerty</code> above. As hackers also have access to this list of common passwords, it's important that none of our users use these passwords!</p>
<p>Let's flag all the passwords in our user database that are among the top 10,000 used passwords.</p>

In [26]:
# Flagging the users with passwords that are common passwords
users$common_password <- ifelse(users$password %in% common_passwords, TRUE, FALSE)

# Counting the number of users using common passwords
sum(users$common_password)

# Taking a look at the 12 first rows
head(users, 12)

[1] 0

id,user_name,password,length,too_short,common_password
<dbl>,<chr>,<chr>,<int>,<lgl>,<lgl>
1,vance.jennings,joobheco,8,FALSE,FALSE
2,consuelo.eaton,0869347314,10,FALSE,FALSE
3,mitchel.perkins,fabypotter,10,FALSE,FALSE
4,odessa.vaughan,aharney88,9,FALSE,FALSE
5,araceli.wilder,acecdn3000,10,FALSE,FALSE
6,shawn.harrington,5278049,7,TRUE,FALSE
7,evelyn.gay,master,6,TRUE,FALSE
8,noreen.hale,murphy,6,TRUE,FALSE
9,gladys.ward,lwsves2,7,TRUE,FALSE


## 5. Passwords should not be common words
<p>Ay ay ay! It turns out many of our users use common passwords, and of the first 12 users there are already two. However, as most common passwords also tend to be short, they were already flagged as being too short. What is the next thing we should check?</p>
<blockquote>
  <p>Verifiers SHALL compare the prospective secrets against a list that contains [...] dictionary words.</p>
</blockquote>
<p>This follows the same logic as before: It is easy for hackers to check users' passwords against common English words and therefore common English words make bad passwords. Let's check our users' passwords against the top 10,000 English words from <a href="https://github.com/first20hours/google-10000-english">Google's Trillion Word Corpus</a>.</p>

In [28]:
# Reading in a list of the 10000 most common words
words <- read_csv('datasets/google-10000-english.txt')

# Flagging the users with passwords that are common words
users$common_word <- ifelse(str_to_lower(users$password) %in% words, TRUE, FALSE)

# Counting the number of users using common words as passwords
sum(users$common_word)

# Taking a look at the 12 first rows
head(users, 12)

Parsed with column specification:
cols(
  the = col_character()
)


[1] 0

id,user_name,password,length,too_short,common_password,common_word
<dbl>,<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>
1,vance.jennings,joobheco,8,FALSE,FALSE,FALSE
2,consuelo.eaton,0869347314,10,FALSE,FALSE,FALSE
3,mitchel.perkins,fabypotter,10,FALSE,FALSE,FALSE
4,odessa.vaughan,aharney88,9,FALSE,FALSE,FALSE
5,araceli.wilder,acecdn3000,10,FALSE,FALSE,FALSE
6,shawn.harrington,5278049,7,TRUE,FALSE,FALSE
7,evelyn.gay,master,6,TRUE,FALSE,FALSE
8,noreen.hale,murphy,6,TRUE,FALSE,FALSE
9,gladys.ward,lwsves2,7,TRUE,FALSE,FALSE


## 6. Passwords should not be your name
<p>It turns out many of our passwords were common English words too! Next up on the NIST list:</p>
<blockquote>
  <p>Verifiers SHALL compare the prospective secrets against a list that contains [...] context-specific words, such as the name of the service, the username, and derivatives thereof.</p>
</blockquote>
<p>Ok, so there are many things we could check here. One thing to notice is that our users' usernames consist of their first names and last names separated by a dot. For now, let's just flag passwords that are the same as either a user's first or last name.</p>

In [30]:
# Extracting first and last names into their own columns
users$first_name <- str_extract(users$user_name, "^\\w+")
users$last_name <- str_extract(users$user_name, "\\w+$")

# Flagging the users with passwords that matches their names
users$uses_name <- ifelse(users$first_name == users$password | users$last_name == users$password, TRUE, FALSE)

# Counting the number of users using names as passwords
sum(users$uses_name)

# Taking a look at the 12 first rows
head(users, 12)

[1] 50

id,user_name,password,length,too_short,common_password,common_word,first_name,last_name,uses_name
<dbl>,<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<lgl>
1,vance.jennings,joobheco,8,FALSE,FALSE,FALSE,vance,jennings,FALSE
2,consuelo.eaton,0869347314,10,FALSE,FALSE,FALSE,consuelo,eaton,FALSE
3,mitchel.perkins,fabypotter,10,FALSE,FALSE,FALSE,mitchel,perkins,FALSE
4,odessa.vaughan,aharney88,9,FALSE,FALSE,FALSE,odessa,vaughan,FALSE
5,araceli.wilder,acecdn3000,10,FALSE,FALSE,FALSE,araceli,wilder,FALSE
6,shawn.harrington,5278049,7,TRUE,FALSE,FALSE,shawn,harrington,FALSE
7,evelyn.gay,master,6,TRUE,FALSE,FALSE,evelyn,gay,FALSE
8,noreen.hale,murphy,6,TRUE,FALSE,FALSE,noreen,hale,FALSE
9,gladys.ward,lwsves2,7,TRUE,FALSE,FALSE,gladys,ward,FALSE


## 7. Passwords should not be repetitive
<p>Milford Hubbard (user number 12 above), what where you thinking!? Ok, so the last thing we are going to check is a bit tricky:</p>
<blockquote>
  <p>verifiers SHALL compare the prospective secrets [so that they don't contain] repetitive or sequential characters (e.g. ‘aaaaaa’, ‘1234abcd’).</p>
</blockquote>
<p>This is tricky to check because what is <em>repetitive</em> is hard to define. Is <code>11111</code> repetitive? Yes! Is <code>12345</code> repetitive? Well, kind of. Is <code>13579</code> repetitive? Maybe not..? To check for <em>repetitiveness</em> can be arbitrarily complex, but here we're only going to do something simple. We're going to flag all passwords that contain 4 or more repeated characters.</p>

In [32]:
# Splitting the passwords into vectors of single characters
split_passwords <-  str_split(users$password, "")

# Picking out the max number of repeat characters for each password
users$max_repeats <- sapply(split_passwords, function(split_password) {
    rle_password <- rle(split_password)
    max(rle_password$lengths)
})


# Flagging the passwords with >= 4 repeats
users$too_many_repeats <- ifelse(users$max_repeats >= 4, TRUE, FALSE)

# Taking a look at the users with too many repeats
users$too_many_repeats

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [37] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [61] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [73] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [85] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [97] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[109] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[121] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[133] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[145] FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[157] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[169] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[181] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[193] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[205] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[217] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[229] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[241] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[253] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[265] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[277] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[289] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[301] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[313] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[325] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[337] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[349] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[361] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[373] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[385] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[397] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[409] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[421] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[433] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[445] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[457] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[469] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[481] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[493] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[505] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[517] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[529] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[541] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[553] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[565] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
[577] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[589] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[601] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[613] FALSE FALSE FALSE 

## 8. All together now!
<p>Now we have implemented all the basic tests for bad passwords suggested by NIST Special Publication 800-63B! What's left is just to flag all bad passwords and maybe to send these users an e-mail that strongly suggests they change their password.</p>

In [34]:
# Flagging all passwords that are bad
users$bad_password <- ifelse((users$too_short == TRUE |
                              users$common_password == TRUE |
                              users$common_word == TRUE |
                              users$uses_name == TRUE |
                              users$too_many_repeats == TRUE), TRUE, FALSE)

# Counting the number of bad passwords
sum(users$bad_password)

# Looking at the first 100 bad passwords
head(users$bad_password == TRUE, 100)

[1] 383

[1] FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE FALSE FALSE  TRUE
 [13] FALSE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE  TRUE FALSE
 [25] FALSE  TRUE  TRUE FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE
 [37] FALSE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE FALSE  TRUE
 [49]  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE  TRUE  TRUE
 [61] FALSE FALSE  TRUE  TRUE  TRUE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE
 [73] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE
 [85]  TRUE FALSE FALSE  TRUE FALSE  TRUE FALSE  TRUE  TRUE FALSE FALSE  TRUE
 [97] FALSE  TRUE FALSE FALSE

## 9. Otherwise, the password should be up to the user
<p>In this notebook, we've implemented the password checks recommended by the NIST Special Publication 800-63B. It's certainly possible to better implement these checks, for example, by using a longer list of common passwords. Also note that the NIST checks in no way guarantee that a chosen password is good, just that it's not obviously bad.</p>
<p>Apart from the checks we've implemented above the NIST is also clear with what password rules should <em>not</em> be imposed:</p>
<blockquote>
  <p>Verifiers SHOULD NOT impose other composition rules (e.g., requiring mixtures of different character types or prohibiting consecutively repeated characters) for memorized secrets. Verifiers SHOULD NOT require memorized secrets to be changed arbitrarily (e.g., periodically).</p>
</blockquote>
<p>So the next time a website or app tells you to "include both a number, symbol and an upper and lower case character in your password" you should send them a copy of <a href="https://pages.nist.gov/800-63-3/sp800-63b.html">NIST Special Publication 800-63B</a>.</p>

In [36]:
# Enter a password that passes the NIST requirements
# PLEASE DO NOT USE AN EXISTING PASSWORD HERE
new_password <- "qwerty123"